In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

In [2]:
# RDD만을 이용한 데이터 추출

In [3]:

movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [4]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [5]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [6]:

movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [7]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [8]:
# Spark SQL 사용해 보기

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [10]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [11]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [12]:
# 2. 데이터 프레임 만들기

In [13]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [14]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [15]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [16]:
df.select('company').show()

+-------+
|company|
+-------+
|   마블|
|     DC|
|     DC|
| 디즈니|
|   마블|
+-------+



In [17]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [18]:
# 년, 월, 일
df.select(df.year, df.month, df.day).show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [19]:
# 2013년 이후 영화
df.filter(df.year >= 2013).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [20]:
# 마블영화, DC 같이 꺼내기
df.filter((df.company == '마블') | (df.company == 'DC')).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [30]:
df.createOrReplaceTempView("movies")

In [31]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [32]:
# 2010년 이후에 개봉한 영화를 조회

query = """

"""
spark.sql(query).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near end of input.(line 3, pos 0)

== SQL ==
^^^


In [ ]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """

"""
spark.sql(query).show()

In [ ]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """

"""
spark.sql(query).show()

In [ ]:

# BETWEEN 특정 데이터와 데이터 사이를 조회

# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8


In [ ]:
# Join 구현하기

In [23]:

attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [24]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [25]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [26]:

att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [27]:
att_df.createOrReplaceTempView("att")

In [28]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [33]:
# df와 join
query = '''
select movies.id, movies.name, movies.company, att.att
from movies
join att ON movies.id = att.id
'''

spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [ ]:
# 데이터 프레임 API

In [ ]:
# select
df.select("*").collect()

In [ ]:
df.select("name", "company").collect()

In [ ]:
df.select(df.name, (df.year-2000).alias("year")).show()

In [ ]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

In [ ]:
df.groupBy().avg().collect()

In [ ]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

In [ ]:
# 회사 별 월 별 영화 개수 정보


In [ ]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

In [ ]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

In [ ]:
spark.stop()
sc.stop()

In [ ]:
ㅌ

In [ ]:
# SQL 최적화

In [34]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [37]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [38]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [39]:
data.createOrReplaceTempView("mobility_data")

In [40]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [41]:
# 스파크 SQL을 사용하는 이유

In [42]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [43]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#281, 'count(1) AS trips#282]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#208 as string),  , -1)[0]], [split(cast(pickup_datetime#208 as string),  , -1)[0] AS pickup_date#281, count(1) AS trips#282L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#206,dispatching_base_num#207,pickup_datetime#208,dropoff_datetime#209,PULocationID#210,DOLocationID#211,SR_Flag#212])
      +- Relation [hvfhs_license_num#206,dispatching_base_num#207,pickup_datetime#208,dropoff_datetime#209,PULocationID#210,DOLocationID#211,SR_Flag#212] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#286], [_groupingexpression#286 AS pickup_date#281, count(1) AS trips#282L]
+- Project [split(cast(pickup_datetime#208 as string),  , -1)[0] AS _groupingexpression#286]
   +- Rel

In [44]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#290]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#289]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#289], [pickup_date#289, count(1) AS trips#290L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#208 as string),  , -1)[0] AS pickup_date#289]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#206,dispatching_base_num#207,pickup_datetime#208,dropoff_datetime#209,PULocationID#210,DOLocationID#211,SR_Flag#212])
            +- Relation [hvfhs_license_num#206,dispatching_base_num#207,pickup_datetime#208,dropoff_datetime#209,PULocationID#210,DOLocationID#211,SR_Flag#212] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#289], [pickup_date#289, count(1) AS tri

In [ ]:
spark.stop()

In [46]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

zone_file = "taxi+_zone_lookup.csv"

In [ ]:
# 운행 데이터프레임 생성, zone 데이터프레임 생성
# 

In [47]:
trip_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/fhvhv_tripdata_2020-03.csv')
trip_df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [48]:
zone_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/taxi+_zone_lookup.csv')
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
# 승차 Location(PULocationID)별 개수 세기

In [49]:
trip_df.groupBy('PULocationID').count().show()

+------------+------+
|PULocationID| count|
+------------+------+
|         148|116205|
|         243| 87431|
|          31|  5285|
|         137| 85552|
|          85| 46120|
|         251|  9080|
|          65| 66622|
|         255|113947|
|          53| 17571|
|         133| 27200|
|          78| 76155|
|         108| 20378|
|         155| 39527|
|         211| 61075|
|         193| 20111|
|          34| 11823|
|         115| 10806|
|         126| 52833|
|         101|  8983|
|          81| 41425|
+------------+------+
only showing top 20 rows



In [ ]:
# 하차 Location(DOLocationID)별 개수 세기

In [50]:
trip_df.groupBy('DOLocationID').count().show()

+------------+------+
|DOLocationID| count|
+------------+------+
|         148| 91601|
|         243| 86795|
|          31|  5526|
|          85| 44509|
|         137| 80098|
|         251|  8525|
|          65| 58888|
|         255|105051|
|          53| 19013|
|         133| 27760|
|          78| 74447|
|         155| 42239|
|         108| 21354|
|         211| 54176|
|         193| 19104|
|          34| 12392|
|         115|  9809|
|         101|  7218|
|         126| 59027|
|          81| 38445|
+------------+------+
only showing top 20 rows



In [ ]:
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num

In [55]:
from pyspark.sql.functions import count, desc

In [61]:
hv0003_df = trip_df.filter(trip_df.hvfhs_license_num == 'HV0003')   # HV0003 운송사업자만 필터링

trip_count_by_location = hv0003_df.groupBy('PULocationID').count()   # 필터링된 데이터에서 PULocationID별로 그룹화하고 개수를 센다

In [62]:
sorted_trip_count = trip_count_by_location.orderBy(desc('count'))   # 결과를 count 컬럼을 기준으로 내림차순 정렬

sorted_trip_count.show()

+------------+------+
|PULocationID| count|
+------------+------+
|          61|163091|
|          76|134198|
|         132|114179|
|          79|112017|
|          37|110150|
|          42|108070|
|         138|104119|
|         244| 97324|
|          89| 95724|
|          39| 94484|
|         231| 94155|
|           7| 92676|
|          17| 90352|
|         161| 90261|
|         225| 88749|
|         234| 88372|
|         230| 86870|
|         188| 84347|
|          35| 82764|
|         168| 82396|
+------------+------+
only showing top 20 rows



In [ ]:
#운송사별 운행 건수 비교

In [63]:
trips_by_hvfhs = trip_df.groupBy('hvfhs_license_num').agg(
    count('hvfhs_license_num').alias('total_trips')
)


trips_by_hvfhs.orderBy(desc('total_trips')).show()

+-----------------+-----------+
|hvfhs_license_num|total_trips|
+-----------------+-----------+
|           HV0003|    9836763|
|           HV0005|    3219535|
|           HV0004|     336606|
+-----------------+-----------+



In [ ]:
#승차 위치 Borough별 운행 건수

In [64]:
# 데이터 조인
trips_with_borough = trip_df.join(zone_df, trip_df.PULocationID == zone_df.LocationID, "inner")

In [65]:
# 조인된 DataFrame에서 'Borough'별로 그룹화하고 운행 건수를 집계
borough_trip_counts = trips_with_borough.groupBy('Borough').agg(
    count('*').alias('trip_count')
)

In [66]:
borough_trip_counts.orderBy(desc('trip_count')).show()

+-------------+----------+
|      Borough|trip_count|
+-------------+----------+
|    Manhattan|   4953140|
|     Brooklyn|   3735764|
|       Queens|   2437383|
|        Bronx|   2086592|
|Staten Island|    178818|
|      Unknown|       845|
|          EWR|       362|
+-------------+----------+



In [ ]:
#서비스 존별 승차/하차 건수

In [67]:
pickup_zone_df = zone_df.select(
    col("LocationID").alias("PULocationID"),
    col("service_zone").alias("pickup_service_zone")
)

In [68]:
dropoff_zone_df = zone_df.select(
    col("LocationID").alias("DOLocationID"),
    col("service_zone").alias("dropoff_service_zone")
)

In [69]:
# 
trips_with_zones = trip_df.join(
    pickup_zone_df,
    on="PULocationID",
    how="inner"
).join(
    dropoff_zone_df,
    on="DOLocationID",
    how="inner"
)

In [70]:
# 승차 서비스 존별 건수 집계
pickup_counts = trips_with_zones.groupBy("pickup_service_zone").agg(
    count(col("PULocationID")).alias("pickup_trips")
)

pickup_counts.orderBy(desc("pickup_trips")).show(truncate=False)

+-------------------+------------+
|pickup_service_zone|pickup_trips|
+-------------------+------------+
|Boro Zone          |9046897     |
|Yellow Zone        |4025190     |
|Airports           |319610      |
|N/A                |845         |
|EWR                |362         |
+-------------------+------------+



In [71]:
# 하차 서비스 존별 건수 집계
dropoff_counts = trips_with_zones.groupBy("dropoff_service_zone").agg(
    count(col("DOLocationID")).alias("dropoff_trips")
)

dropoff_counts.orderBy(desc("dropoff_trips")).show(truncate=False)

+--------------------+-------------+
|dropoff_service_zone|dropoff_trips|
+--------------------+-------------+
|Boro Zone           |8885136      |
|Yellow Zone         |3643787      |
|Airports            |411156       |
|N/A                 |387759       |
|EWR                 |65066        |
+--------------------+-------------+



In [ ]:
# 질문 만들어보기 3개씩

In [72]:
# - 승객이 가장 많이 승차한 지역ID TOP10
pickup_counts = trip_df.groupBy("PULocationID").agg(
    count("*").alias("trip_count")
)

top_10_pickup_locations = pickup_counts.orderBy(desc("trip_count")).limit(10)
top_10_pickup_locations.show()

+------------+----------+
|PULocationID|trip_count|
+------------+----------+
|          61|    222094|
|          79|    183821|
|          76|    168311|
|         132|    163734|
|         138|    155876|
|          37|    155388|
|          42|    143389|
|         231|    135712|
|         234|    132693|
|         161|    128751|
+------------+----------+



In [73]:
# - 승객이 가장 많이 하차한 지역ID TOP10
pickdown_counts = trip_df.groupBy("DOLocationID").agg(
    count("*").alias("trip_count")
)

top_10_pickdown_locations = pickdown_counts.orderBy(desc("trip_count")).limit(10)
top_10_pickdown_locations.show()

+------------+----------+
|DOLocationID|trip_count|
+------------+----------+
|         265|    387758|
|          61|    224476|
|         132|    216213|
|         138|    194943|
|          76|    172649|
|          37|    158832|
|          79|    147732|
|          42|    136729|
|         225|    128945|
|          17|    128152|
+------------+----------+



In [78]:
# - 승차, 하차 지역ID 동일한 지역의 개수
from pyspark.sql import Row

distinct_same_location_count = trip_df.filter(col('PULocationID') == col('DOLocationID'))\
                                      .select('PULocationID').distinct().count()

result_df = spark.createDataFrame(
    [Row(count_of_locations=distinct_same_location_count)]
)

result_df.show()

+------------------+
|count_of_locations|
+------------------+
|               258|
+------------------+

